In [1]:
stableStringify = (val) => {
  if (Array.isArray(val)) return `[${val.map(stableStringify).join(',')}]`;
  if (val && typeof val === 'object' && !(val instanceof Date)) {
    return `{${Object.keys(val).sort().map(key => `${JSON.stringify(key)}:${stableStringify(val[key])}`).join(',')}}`;
  }
  return JSON.stringify(val);
};
deepEqual = (a, b) => {
  if (a === b) return true;
  const aObj = a && typeof a === 'object';
  const bObj = b && typeof b === 'object';
  if (!aObj && !bObj) return Object.is(a, b);
  return stableStringify(a) === stableStringify(b);
};
formatVal = (val) => (typeof val === 'string' ? `'${val}'` : stableStringify(val));
assert = (fn, expected, label) => {
  try {
    const actual = fn();
    const pass = deepEqual(actual, expected);
    if (pass) {
      console.log('✓', label, '->', actual);
    } else {
      console.error('✗', label, 'expected:', formatVal(expected), 'actual:', formatVal(actual));
    }
  } catch (err) {
    console.error('✗', label, 'threw:', err);
  }
};

[Function: assert]

In [2]:
nameCache = new Map()
kebabToCamel = s => {
  if (!s) return s;
  let p = s.indexOf('-');
  if (p === -1) return s;
  if (nameCache.has(s)) return nameCache.get(s);
  let result = s.slice(0, p);
  while (p != -1 && ++p < s.length) {
    if (s[p] == '-') continue;
    result += s[p].toUpperCase();
    if (++p < s.length)
      result += s.slice(p, (p = s.indexOf('-', p)) == -1 ? s.length : p);
  }
  nameCache.set(s, result);
  return result;
};

assert(() => kebabToCamel('foo-bar'), 'fooBar', 'basic case');
assert(() => kebabToCamel('-bar'), 'Bar', 'lead single');
assert(() => kebabToCamel('bar-'), 'bar', 'trail single');
assert(() => kebabToCamel('multi-part-key'), 'multiPartKey', 'multi part');
assert(() => kebabToCamel('-'), '', 'single dash');
assert(() => kebabToCamel('--'), '', 'multi dashes only');
assert(() => kebabToCamel('--leading--dashes'), 'LeadingDashes', 'leading dashes');
assert(() => kebabToCamel('trailing--dashes-'), 'trailingDashes', 'trailing dashes');

✓ basic case -> fooBar
✓ lead single -> Bar
✓ trail single -> bar
✓ multi part -> multiPartKey
✓ single dash -> 
✓ multi dashes only -> 
✓ leading dashes -> LeadingDashes
✓ trailing dashes -> trailingDashes


In [3]:
// Returns the index of the first found char (from chars) in the s, or -1 if none found,
// then you can check s[retunedIndex] to see which char it is.
indexFirst = (s, chars, pos = 0) => {
  let i, first = s.length
  for (let c of chars) if ((i = s.indexOf(c, pos)) != -1 && i < first) first = i
  return first === s.length ? -1 : first
}

assert(() => indexFirst('abcdefg', ['x', 'y', 'z']), -1, 'none found');
assert(() => indexFirst('abcdefg', ['x', 'c', 'z']), 2, 'one found');
assert(() => indexFirst('abcdefgabc', ['a', 'b', 'c'], 3), 7, 'multiple found with pos');
assert(() => indexFirst('abcdefg', ['a'], 10), -1, 'pos out of range');

✓ none found -> -1
✓ one found -> 2
✓ multiple found with pos -> 7
✓ pos out of range -> -1


In [22]:
// @WIP using +#tpl-id instead of #tpl-id to be predictable in presence of other selectors
MOD='^', TARG=':', TRIG='@', STAT='?', ADD='+', REM='~';
ITEMS = [MOD, TARG, TRIG, STAT, ADD, REM];

parseAttrName = (n, t, p='data-'.length) => {
  let ts = [], mods
  while (true) {
    if ((p = n.indexOf(t, p)) == -1) break
    let end = indexFirst(n, ITEMS, ++p) // until next item or end
    if (end == p) { ts.push(''); continue; } // empty t
    if (end == -1) { ts.push(n.slice(p)); p = -1; break; } // last t
    let it = n.slice(p, end)
    if (t == MOD || n[end] != MOD) { ts.push(it); p = end; continue; }
    [mods, p] = parseAttrName(n, MOD, end)
    ts.push({it, mods})
    if (p == -1) break
  }
  return [ts, p];
};

assert(() => parseAttrName('data-sub:', TARG), [[''], -1], 'data attr basic');
assert(() => parseAttrName('data-sub:xxx~', TARG), [['xxx'], -1], 'name only');
assert(() => parseAttrName('data-sub:xxx:', TARG), [['xxx', ''], -1], 'name and empty');
assert(() => parseAttrName('data-sub::', TARG), [['', ''], -1], '2 empties');
assert(() => parseAttrName('data-sub:foo^', TARG),  [[{"it":"foo","mods":[""]}],-1], 'empty mod');
assert(() => parseAttrName('data-sub:foo^^', TARG), [[{"it":"foo","mods":["",""]}], -1], 'empty mods');


✓ data attr basic -> [ [ '' ], -1 ]
✓ name only -> [ [ 'xxx' ], -1 ]
✓ name and empty -> [ [ 'xxx', '' ], -1 ]
✓ 2 empties -> [ [ '', '' ], -1 ]
✓ empty mod -> [ [ { it: 'foo', mods: [Array] } ], -1 ]
✓ empty mods -> [ [ { it: 'foo', mods: [Array] } ], -1 ]
